# Data Wrangling Assignment

### First step is to import the necessary libraries and load the dataset


In [1]:
#import the Pandas Library
import pandas as pd


In [2]:
#Loading the Netflix Dataset
df = pd.read_csv("netflix_titles.csv")

### Data Discovery

In [3]:
#Quick overview of the DataSet
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8807 entries, 0 to 8806
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   show_id       8807 non-null   object
 1   type          8807 non-null   object
 2   title         8807 non-null   object
 3   director      6173 non-null   object
 4   cast          7982 non-null   object
 5   country       7976 non-null   object
 6   date_added    8797 non-null   object
 7   release_year  8807 non-null   int64 
 8   rating        8803 non-null   object
 9   duration      8804 non-null   object
 10  listed_in     8807 non-null   object
 11  description   8807 non-null   object
dtypes: int64(1), object(11)
memory usage: 825.8+ KB


##### The non-uniformity in the number of values in the category suggests that later on, we might see missing values in director, cast, country, date_added,rating and duration.

#### Shape of the dataset.This will print the number of rows and columns

In [5]:
print("Shape of the dataset(Rows,Columns): ", df.shape)


Shape of the dataset(Rows,Columns):  (8807, 12)


#### Listing the name of the columns (header names)

In [8]:
print("Columns in the dataset: \n", df.columns)

Columns in the dataset: 
 Index(['show_id', 'type', 'title', 'director', 'cast', 'country', 'date_added',
       'release_year', 'rating', 'duration', 'listed_in', 'description'],
      dtype='object')


#### Checking for null values


In [9]:
print("Missing values per columns:\n", df.isnull().sum())

Missing values per columns:
 show_id            0
type               0
title              0
director        2634
cast             825
country          831
date_added        10
release_year       0
rating             4
duration           3
listed_in          0
description        0
dtype: int64


##### As we can see , several columns seem to have missing values: director, cast, country, date_added, rating, duration. This coincides with the non-uniformity seen earlier.

### Checking for duplicates

In [10]:
print("Number of duplicate rows: ", df.duplicated().sum())

Number of duplicate rows:  0


##### We seem to have no duplicates in values in the dataset

## Data Structuring


#### Converting the data_added column to datetime

In [11]:
df["date_added"]=pd.to_datetime(df["date_added"],format='mixed')

#### Separate 'duration' into numeric value and unit

In [ ]:
df[['duration_value', 'duration_unit']] = df['duration'].str.extract(r'(\d+)\s*(\w+)')

#### Convert duration_value to numeric

In [16]:
df['duration_value'] = pd.to_numeric(df['duration_value'])

#### View Resulting columns



In [18]:
print(df[['duration_value', 'duration_unit']])

      duration_value duration_unit
0               90.0           min
1                2.0       Seasons
2                1.0        Season
3                1.0        Season
4                2.0       Seasons
...              ...           ...
8802           158.0           min
8803             2.0       Seasons
8804            88.0           min
8805            88.0           min
8806           111.0           min

[8807 rows x 2 columns]


## Data Cleaning

##### From the discovery phase, we know that the dataset has no duplicate values and that some columns have missing values

In [21]:
# Dropping the unnecessary columns
df = df.drop(columns=['description'])

##### For certain columns, we will fill them through imputation. Imputation is a  method where missing values are filled based on the information from other related columns

In [23]:
#Impute Director values by using relationship between cast and director
# List of Director-Cast pairs and the number of times they appear
df['dir_cast'] = df['director'] + '---' + df['cast']
counts = df['dir_cast'].value_counts() #counts unique values
filtered_counts = counts[counts >= 3] #checks if repeated 3 or more times
filtered_values = filtered_counts.index #gets the values i.e. names
lst_dir_cast = list(filtered_values) #convert to list
dict_direcast = dict()
for i in lst_dir_cast :
    director,cast = i.split('---')
    dict_direcast[director]=cast
for i in range(len(dict_direcast)):
    df.loc[(df['director'].isna()) & (df['cast'] == list(dict_direcast.items())[i][1]),'director'] = list(dict_direcast.items())[i][0]


In [24]:
#Assign Not Given to all other director fields
df.loc[df['director'].isna(),'director']='Not Given'

In [26]:
#Use directors to fill all missing countries
directors = df['director']
countries = df['country']
#pair each director with their country use zip() to get an iterator of tuples
pairs = zip(directors, countries)
# Convert the list of tuples into a dictionary
dir_cntry = dict(list(pairs))


In [28]:
# Director matched to Country values used to fill in null country values
for i in range(len(dir_cntry)):
    df.loc[(df['country'].isna()) & (df['director'] == list(dir_cntry.items())[i][0]),'country'] = list(dir_cntry.items())[i][1]


In [29]:
# Assign Not Given to all other country fields
df.loc[df['country'].isna(),'country'] = 'Not Given'


In [31]:
# Assign Not Given to all other fields
df.loc[df['cast'].isna(),'cast'] = 'Not Given'

In [32]:
# dropping other row records that are null
df.drop(df[df['date_added'].isna()].index,axis=0,inplace=True)
df.drop(df[df['rating'].isna()].index,axis=0,inplace=True)
df.drop(df[df['duration'].isna()].index,axis=0,inplace=True)

### Errors

In [33]:
# check if there are any added_dates that come before release_year
import datetime as dt
sum(df['date_added'].dt.year < df['release_year'])
df.loc[(df['date_added'].dt.year < df['release_year']),['date_added','release_year']]


,date_added,release_year
1551,2020-12-14,2021
1696,2020-11-15,2021
2920,2020-02-13,2021
3168,2019-12-06,2020
3287,2019-11-13,2020
3369,2019-10-25,2020
3433,2019-10-11,2020
4844,2018-05-30,2019
4845,2018-05-29,2019
5394,2017-07-01,2018


In [34]:
# sample some of the records and check that they have been accurately replaced
df.iloc[[1551,1696,2920,3168]]


,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,duration_value,duration_unit,dir_cast
1551,s1552,TV Show,Hilda,Not Given,"Bella Ramsey, Ameerah Falzon-Ojo, Oliver Nelso...","United Kingdom, Canada, United States",2020-12-14,2021,TV-Y7,2 Seasons,Kids' TV,2.0,Seasons,NaN
1696,s1697,TV Show,Polly Pocket,Not Given,"Emily Tennant, Shannon Chan-Kent, Kazumi Evans...","Canada, United States, Ireland",2020-11-15,2021,TV-Y,2 Seasons,Kids' TV,2.0,Seasons,NaN
2920,s2921,TV Show,Love Is Blind,Not Given,"Nick Lachey, Vanessa Lachey",United States,2020-02-13,2021,TV-MA,1 Season,"Reality TV, Romantic TV Shows",1.0,Season,NaN
3168,s3169,TV Show,Fuller House,Not Given,"Candace Cameron Bure, Jodie Sweetin, Andrea Ba...",United States,2019-12-06,2020,TV-PG,5 Seasons,TV Comedies,5.0,Seasons,NaN


In [35]:
#Confirm that no more release_year inconsistencies
sum(df['date_added'].dt.year < df['release_year'])

14

### Saving and Exporting

In [38]:
df.to_csv('cleaned_netflix.csv', index=False)